In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('mlib').getOrCreate()

In [3]:
df = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [5]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
# [Age, Experience] ----> new feature ----> independent feature

In [8]:
from pyspark.ml.feature import VectorAssembler

In [11]:
# feature colum will be form of age, experience -> form of 2 column list values
featureassembler = VectorAssembler(inputCols=['age','Experience'], outputCol='Independent Features')

In [12]:
output=featureassembler.transform(df)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
finaldata = output.select('Independent Features', 'Salary') #-> Salary->Dependent feature

In [17]:
finaldata.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
#train test split
train, test = finaldata.randomSplit([0.75,0.25])


In [21]:
#linear regression 
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor  = regressor.fit(train)

In [22]:
regressor.coefficients

DenseVector([-298.2456, 1771.9298])

In [23]:
regressor.intercept

21561.403508772833

In [24]:
# prediction

y_hat = regressor.evaluate(test)

In [25]:
y_hat.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17070.175438596598|
|          [30.0,8.0]| 25000|26789.473684210512|
+--------------------+------+------------------+



In [26]:
y_hat.meanAbsoluteError, y_hat.meanSquaredError

(1929.824561403555, 3743921.2065252806)